In [5]:
# importing modules
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from modules.library import documentRetrieval as dr
from modules.library import queryExp as qe
from modules.library.postgresql import PostgresQL
pg = PostgresQL() 


C:\Users\sarab\Miniconda3\envs\envirolens\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [6]:
from importlib import reload
reload(dr)

<module 'modules.library.documentRetrieval' from 'C:\\Users\\sarab\\work\\try\\enviroLENS\\word-embeddings\\modules\\library\\documentRetrieval.py'>

SyntaxError: invalid syntax (<ipython-input-6-cab7617bd286>, line 1)

In [7]:
# importing word vectors
from gensim.models import KeyedVectors
wiki_en_align = './../data/fasttext/wiki.en.align.vec' #'../../data/fasttext/wiki.en.align.vec'
# get fasttext wiki embeddings for english
wv_wiki_en = KeyedVectors.load_word2vec_format(wiki_en_align)
print('english words {}'.format(len(list(wv_wiki_en.vocab.keys()))))

C:\Users\sarab\Miniconda3\envs\envirolens\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


english words 2519370


In [8]:
# prepare stopword list
from nltk.corpus   import stopwords
from nltk.tokenize import word_tokenize
stopWords = stopwords.words('english')

In [9]:

pg.connect(database="envirolens", user="postgres", password="dbpass") #"eurlex_env_only" "solata.2018"

In [10]:
def db_query(query_words):
    """ Iz baze vrne seznam slovarjev, ki vsebujejo dokumente (tekst) in njihove IDje, ki v tekstu vsebujejo vsaj eno od besed iz seznama(list) query_words"""
    output = '|'.join(query_words)
    SQL = """
            SELECT document_id, fulltext_cleaned FROM documents
            WHERE to_tsvector('english', fulltext_cleaned) @@ to_tsquery('english',""" + '\''+ output + '\'' + """);"""
    documents = pg.execute(SQL)
    return(documents)

In [22]:
tokenized = qe.tokenize('water pollution underground', stopWords)
ext = qe.extend_tokens(tokenized, wv_wiki_en)
candidates = qe.pre_retrieval_KNN('water pollution underground', 5, wv_wiki_en, 10, stopWords,extension=False)

In [84]:
qe.pre_retrieval_KNN('fishing and pollution', 15, wv_wiki_en, 30, stopWords,extension=False)

['pollutions',
 '—fishing',
 'shellfishing',
 '‘fishing',
 'earpollution',
 'flwfishing',
 'pollution,',
 'fishing,\u3000',
 '#pollution',
 'biopollution',
 'fishings',
 'polluting',
 'billfishing',
 'sollution',
 'gamefishing',
 'pollut',
 'fishing,',
 'antipollution',
 'pollutants',
 'codfishing']

In [19]:
tokenized = ["pollution","fish"]
docs = db_query(tokenized) 
#premisli kaj (tokens, ext, candidates) das v db_query (I guess vse? --> ce dela na osnovnem queryju dovolj dobro pusti osnovni, drugace prevec dokumentov)

In [11]:
print(len(docs))

13771


In [12]:
docs[0]

{'document_id': 1000002956,
 'fulltext_cleaned': '7 12 2018 en official journal european union c 443 9 minutes sitting wednesday 20 june 2018 2018 c 443 04 contents 1 summary report workshop 9 2 report economic social partners 9 3 report youth conference 10 4 statement carl b greenidge vice president minister foreign affairs guyana 10 5 statement ekaterina zaharieva deputy prime minister judicial reform minister foreign affairs bulgaria president office eu council 10 6 question time council 10 7 debate council — ‘catch eye’ 11 8 report 15th regional meeting held nairobi kenya east african region 11 13 april 2018 — report co presidents 11 9 approval minutes afternoon sitting monday 18 june 2018 morning afternoon sittings tuesday 19 june 2018 11 10 vote motions resolution included reports submitted three standing committees 12 11 vote urgent motions resolution 13 12 business 13 13 date place 36th session joint parliamentary assembly 13 annex alphabetical list members joint parliamentary 

In [11]:
#ce vzame cas popravi v metriki
def change_dict_structure(dict_list):
    """Takes list of dicts from db_query and changes to dict with key=id, value = text (used for metrices)."""
    texts = {}
    for dic in dict_list:
        doc_id = dic.get('document_id')
        text = dic.get('fulltext_cleaned')
        texts.update({doc_id: text})
    return texts

In [20]:
texts_dict = change_dict_structure(docs)


In [ ]:
#texts_dict.get(2)

In [ ]:
multiply0 = dr.probability_score(tokenized,texts_dict, dr.probability_multiply,10)
multiply1 = dr.probability_score(tokenized+ext,texts_dict, dr.probability_multiply,10)
multiply2 = dr.probability_score(tokenized+ext+candidates,texts_dict, dr.probability_multiply,10)
# takes also probability_function probability_sum_weight, but doesnt give final result (used in probability_score_sum_weights)
    # top_expansion are the onesthat are not counted as original query, usually ext ((extension by summation of 2 consecutive words) counts as original query
#     """Assigns score to documents based on probability_function metric.
#     Args:
#         tokens (list): List of tokens (tokenized query). If needed also extension (extension by summation of 2 consecutive words and/or KNN candidates).
#         texts (dict):  Keys represent document ids, values are document text. 
#         probability_function (function): Metric function that calculates document relavance. Functions: probability_multiply, probability_sum. Require only first 4 arguments.
#         n (int): Number of returned tuples, sorted by highest scores.
#         top_expansion (list): List of expanded words. Usually candidates (kNN expansion), then have to be also in tokens.
#         alpha (float): Number between 0 and 1. Weight that emphasizes the difference between original query words and expansions. 
#                        For alpha 0.5 all words have same weights (but not same values!), for alpha 1 expansion words have value 0. 
#                        For alpha -1 values equal to cosine similarity to query words. 
#         wv (Word2VecKeyedVectors): Word embeddings.
#     Returns:
#         document_probability (list): Tuples of document ids and scores that measure document relavance. Returns n tuples with highest score.
#     """


In [ ]:
multiply0 

In [ ]:
dr.probability_score_sum_weights(original_tokens, top_expansion, texts,n, alpha, wv)
# """As probability_score only weighted.
#         Args:
#         original_tokens(list): List of strings. Tokenized original query. Usually also extension (extension by summation of 2 consecutive words), but not top_expansion (kNN candidates)
#         top_expansion (list): List of expanded words. Usually candidates (kNN expansion).
#         texts (dict):  Keys represent document ids, values are document text.
#         n (int): Number of returned tuples, sorted by highest scores.
#         alpha (float): Number between 0 and 1. Weight that emphasizes the difference between original query words and expansions. 
#                        For alpha 0.5 all words have same weights (but not same values!), for alpha 1 expansion words have value 0. 
#                        For alpha -1 values equal to cosine similarity to query words. 
#         wv (Word2VecKeyedVectors): Word embeddings.
#     Returns:
#         document_score (list): Tuples of document ids and scores that measure document relavance. Returns n tuples with highest score.
#     """

In [ ]:
dr.probability_score_sum_weights(tokenized+ext, candidates, texts_dict,5, 0.6, wv_wiki_en)

In [ ]:
dr.tfidf_score(tokens, texts, tfidf_function,n, *args)
#     """Assigns score to documents based on tfidf_function metric.
#     Args:
#         tokens (list): List of tokens (tokenized query). If needed also extension (extension by summation of 2 consecutive words).For tfidf_sum metric can also add kNN candidates.
#         texts (dict):  Keys represent document ids, values are document text. 
#         probability_function (function): Metric function that calculates document relavance. Functions: tfidf_sum; require only first 4 arguments, tfidf_sum_weight; require all arguments.
#         n (int): Number of returned tuples, sorted by highest scores.
#         top_expansion (list): List of expanded words. Usually candidates (kNN expansion).
#         alpha (float): Number between 0 and 1. Weight that emphasizes the difference between original query words and expansions. 
#                        For alpha 0.5 all words have same weights (but not same values!), for alpha 1 expansion words have value 0. 
#                        For alpha -1 values equal to cosine similarity to query words. 
#         wv (Word2VecKeyedVectors): Word embeddings.
#     Returns:
#         document_probability (list): Tuples of document ids and scores that measure document relavance. Returns n tuples with highest score.
#         not_appear (list): List of words that did not occure in any document.
#     """

In [12]:
import time

In [23]:
start_time = time.time()
dr.tfidf_score(tokenized+ext, texts_dict, dr.tfidf_sum,10)
print(time.time() - start_time)

2.2841923236846924


In [63]:
tokens = ['underground', 'fish', 'water', 'pollution']

In [72]:
test_list= ['seawater',
 'fishwater',
 'groundwaters',
 'sewage',
 'groundwater','potable',
 'undergrounding',
 'undergrounders',
 'ρwater',
 'wastewater']

In [73]:
len(test_list)

10

In [70]:

dcs = db_query(tokens) 
document_list = change_dict_structure(dcs)


In [74]:
start_time = time.time()
dr.tfidf_score(tokens+test_list, document_list, dr.tfidf_sum,10)
print(time.time() - start_time)

8.35601806640625


In [37]:
start2 = time.time()
dr.tfidf_score(tokenized+ext, texts_dict, dr.tfidf_sum_weight,10, candidates, 0.6, wv_wiki_en)
end2 = time.time()
print(end2 - start2)


84.60514283180237


In [20]:
def tfidf_score_str(tokens,texts,tfidf_function_name,m,*args):
    """Takes function name as input, returns function"""
    if tfidf_function_name == 'tfidf_sum':
            return dr.tfidf_score(tokens,texts,dr.tfidf_sum,m)
    else:
        print("Error, different fn name")

In [40]:
start = time.time()
sc = tfidf_score_str(tokenized, texts_dict, "tfidf_sum",10)
end = time.time()
print(end - start)
# s stevilom besed (+ext) se cas poveca

0.8638277053833008


In [22]:
sc

[(1000091525, 0.00826585345771757),
 (1000097317, 0.0075142127850723614),
 (1000094934, 0.007331187304515795),
 (1000000045, 0.006758104887364456),
 (1000003451, 0.006355128936113985),
 (1000093531, 0.006151154273753995),
 (1000059173, 0.005921035429548822),
 (1000097318, 0.005864949843612636),
 (1000094663, 0.005702892018986497),
 (1000022091, 0.005608434180162704)]

In [ ]:
# convert results from metric functions to dictionary
# def results_to_dictionary(results):
#     results_dict = {}
#     for tupl in results:
#         results_dict.update({tupl[0]:tupl[1]})
#     return(results_dict)
        
        
    

In [ ]:
# TREC setting


In [83]:
import json
# import documents
directory = '/Users/sarab/work/try/TREC/processed_data/wFT/' 
text = {}
for filename in os.listdir(directory):
    with open('C:\\Users\\sarab\\work\\try\\TREC\\processed_data\\wFT\\'+filename, 'r') as file:
        for line in file: 
            dicti = json.loads(line)
            text = {**text, **dicti}
# import questions
topics = {} 
with open('C:\\Users\\sarab\\work\\try\\TREC\\processed_data\\titles_topics', 'r') as file:
        for line in file: 
            dicti = json.loads(line)
            topics = {**topics, **dicti}



FileNotFoundError: [WinError 3] The system cannot find the path specified: '/Users/sarab/work/try/TREC/processed_data/wFT/'

In [79]:
# creates pytrec_eval form of results and writes a file
# args: name - name of dictionary, we are creating
#       topics_dict  - dict of question numbers and questions
#       results - metric results we want to convert
def make_dict(topics_dict):
    main_dicti = {}
    for key in topics_dict:
        main_dicti[key] = {}
    return main_dicti

def store_to_dict(this_key, results, main_dicti):
    for tupl in results:
        main_dicti[this_key][tupl[0]] = tupl[1]
    return main_dicti
        
def write_dict(name,main_dicti):  
    file = open(name,"w+") 
    dict_to_write = json.dumps(main_dicti)
    file.write(dict_to_write +"\n")
    file.close() 

    

In [80]:
def to_dict(name, this_key,topics_dict,results):
    try:
        
        with open(name, 'r+') as file:
            for line in file: 
                dicti = json.loads(line)
               
            for tupl in results:
                dicti[this_key][tupl[0]] = tupl[1]

            dict_to_write = json.dumps(dicti)
            file.seek(0)
            file.write(dict_to_write +"\n")
            file.truncate()
            file.close() 
           
            
        
        
    except:
        main_dicti = {}
        for key in topics_dict:
            main_dicti[key] = {}
        for tupl in results:
            main_dicti[this_key][tupl[0]] = tupl[1]
    
        file = open(name,"w+") 
        dict_to_write = json.dumps(main_dicti)
        file.write(dict_to_write +"\n")
        file.close() 
      


In [ ]:
# testing metrics  

import time

start = time.time()

main_dicti = {}
for key in topics:
    main_dicti[key] = {}

for key in topics:
    print(key)
    tokenized = qe.tokenize(topics.get(key), stopWords)
    ext = qe.extend_tokens(tokenized, wv_wiki_en)
    candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=False)
    ext_candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=True)
    
    multiply0 = dr.probability_score(tokenized,text, dr.probability_multiply,0)
    for tupl in multiply0:
        main_dicti[key][tupl[0]] = tupl[1]
        
file = open('multiply0',"w+") 
dict_to_write = json.dumps(main_dicti)
file.write(dict_to_write +"\n")
file.close()

end = time.time()
print(end - start)



In [ ]:
import time

start = time.time()

main_dicti1 = {}
for key in topics:
    main_dicti1[key] = {}
    
main_dicti2 = {}
for key in topics:
    main_dicti2[key] = {}
    
main_dicti3 = {}
for key in topics:
    main_dicti3[key] = {}
    
    

for key in topics:
 
    tokenized = qe.tokenize(topics.get(key), stopWords)
    ext = qe.extend_tokens(tokenized, wv_wiki_en)
    candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=False)
    ext_candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=True)
    
        
    multiply1 = dr.probability_score(tokenized+ext,text, dr.probability_multiply,0)
    for tupl in multiply1:
        main_dicti1[key][tupl[0]] = tupl[1]
    
    multiply2 = dr.probability_score(tokenized+ext+ext_candidates,text, dr.probability_multiply,0)
    for tupl in multiply2:
        main_dicti2[key][tupl[0]] = tupl[1]
        
    multiply3 = dr.probability_score(tokenized+candidates,text, dr.probability_multiply,0)
    for tupl in multiply3:
        main_dicti3[key][tupl[0]] = tupl[1]
        
file = open('t_multiply1',"w+") 
dict_to_write = json.dumps(main_dicti1)
file.write(dict_to_write +"\n")
file.close()

file = open('t_multiply2',"w+") 
dict_to_write = json.dumps(main_dicti2)
file.write(dict_to_write +"\n")
file.close()

file = open('t_multiply3',"w+") 
dict_to_write = json.dumps(main_dicti3)
file.write(dict_to_write +"\n")
file.close()

end = time.time()
print(end - start)

In [ ]:
import time

start = time.time()

main_dicti0 = {}
for key in topics:
    main_dicti0[key] = {}

main_dicti1 = {}
for key in topics:
    main_dicti1[key] = {}
    
main_dicti2 = {}
for key in topics:
    main_dicti2[key] = {}
    
main_dicti3 = {}
for key in topics:
    main_dicti3[key] = {}
    
    

for key in topics:

    tokenized = qe.tokenize(topics.get(key), stopWords)
    ext = qe.extend_tokens(tokenized, wv_wiki_en)
    candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=False)
    ext_candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=True)
    

    
    sum0 = dr.probability_score(tokenized,text, dr.probability_sum,0)
    for tupl in sum0:
        main_dicti0[key][tupl[0]] = tupl[1]
    
    sum1 = dr.probability_score(tokenized+ext,text, dr.probability_sum,0)
    for tupl in sum1:
        main_dicti1[key][tupl[0]] = tupl[1]
        
    sum2 = dr.probability_score(tokenized+ext+ext_candidates,text, dr.probability_sum,0)
    for tupl in sum2:
        main_dicti2[key][tupl[0]] = tupl[1]
        
    sum3 = dr.probability_score(tokenized+candidates,text, dr.probability_sum,0)
    for tupl in sum3:
        main_dicti3[key][tupl[0]] = tupl[1]
        
file = open('t_sum0',"w+") 
dict_to_write = json.dumps(main_dicti0)
file.write(dict_to_write +"\n")
file.close()
        
file = open('t_sum1',"w+") 
dict_to_write = json.dumps(main_dicti1)
file.write(dict_to_write +"\n")
file.close()

file = open('t_sum2',"w+") 
dict_to_write = json.dumps(main_dicti2)
file.write(dict_to_write +"\n")
file.close()

file = open('t_sum3',"w+") 
dict_to_write = json.dumps(main_dicti3)
file.write(dict_to_write +"\n")
file.close()

end = time.time()
print(end - start)

In [19]:
import time
start = time.time()

main_dicti0 = {}
for key in topics:
    main_dicti0[key] = {}

main_dicti1 = {}
for key in topics:
    main_dicti1[key] = {}
    
main_dicti2 = {}
for key in topics:
    main_dicti2[key] = {}
    
main_dicti3 = {}
for key in topics:
    main_dicti3[key] = {}
    
    

for key in topics:
  
    tokenized = qe.tokenize(topics.get(key), stopWords)
    ext = qe.extend_tokens(tokenized, wv_wiki_en)
    candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=False)
    ext_candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=True)
    
    
    tfidf_sum0 = dr.tfidf_score(tokenized, text, dr.tfidf_sum,0)
    for tupl in tfidf_sum0:
        main_dicti0[key][tupl[0]] = tupl[1]
    
    tfidf_sum1 = dr.tfidf_score(tokenized+ext, text, dr.tfidf_sum,0)
    for tupl in tfidf_sum1:
        main_dicti1[key][tupl[0]] = tupl[1]
        
    tfidf_sum2 = dr.tfidf_score(tokenized+ext+ext_candidates, text, dr.tfidf_sum,0)
    for tupl in tfidf_sum2:
        main_dicti2[key][tupl[0]] = tupl[1]
        
    tfidf_sum3 = dr.tfidf_score(tokenized+candidates, text, dr.tfidf_sum,0)
    for tupl in tfidf_sum3:
        main_dicti3[key][tupl[0]] = tupl[1]
        
file = open('t_tfidf_sum0',"w+") 
dict_to_write = json.dumps(main_dicti0)
file.write(dict_to_write +"\n")
file.close()
        
file = open('t_tfidf_sum1',"w+") 
dict_to_write = json.dumps(main_dicti1)
file.write(dict_to_write +"\n")
file.close()

file = open('t_t_tfidf_sum2',"w+") 
dict_to_write = json.dumps(main_dicti2)
file.write(dict_to_write +"\n")
file.close()

file = open('t_tfidf_sum3',"w+") 
dict_to_write = json.dumps(main_dicti3)
file.write(dict_to_write +"\n")
file.close()

end = time.time()
print(end - start)

3722.501778125763


In [20]:
start = time.time()

main_dicti0 = {}
for key in topics:
    main_dicti0[key] = {}

main_dicti1 = {}
for key in topics:
    main_dicti1[key] = {}
    
main_dicti2 = {}
for key in topics:
    main_dicti2[key] = {}
    
main_dicti3 = {}
for key in topics:
    main_dicti3[key] = {}
    
main_dicti4 = {}
for key in topics:
    main_dicti4[key] = {}
    
    

for key in topics:

    tokenized = qe.tokenize(topics.get(key), stopWords)
    ext = qe.extend_tokens(tokenized, wv_wiki_en)
    candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=False)
    ext_candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=True)
    

    wsum01 = dr.probability_score_sum_weights(tokenized, candidates, text,0, -1, wv_wiki_en)
    for tupl in wsum01:
        main_dicti0[key][tupl[0]] = tupl[1]
        
    wsum006 = dr.probability_score_sum_weights(tokenized, candidates, text,0, 0.6, wv_wiki_en)
    for tupl in wsum006:
        main_dicti1[key][tupl[0]] = tupl[1]
        
    wsum007 = dr.probability_score_sum_weights(tokenized, candidates, text,0, 0.7, wv_wiki_en)
    for tupl in wsum007:
        main_dicti2[key][tupl[0]] = tupl[1]
        
    wsum008 = dr.probability_score_sum_weights(tokenized, candidates, text,0, 0.8, wv_wiki_en)
    for tupl in wsum008:
        main_dicti3[key][tupl[0]] = tupl[1]
        
    wsum009 = dr.probability_score_sum_weights(tokenized, candidates, text,0, 0.9, wv_wiki_en)
    for tupl in wsum009:
        main_dicti4[key][tupl[0]] = tupl[1]
        
        
file = open('t_wsum01',"w+") 
dict_to_write = json.dumps(main_dicti0)
file.write(dict_to_write +"\n")
file.close()
        
file = open('t_wsum006',"w+") 
dict_to_write = json.dumps(main_dicti1)
file.write(dict_to_write +"\n")
file.close()

file = open('t_wsum007',"w+") 
dict_to_write = json.dumps(main_dicti2)
file.write(dict_to_write +"\n")
file.close()

file = open('t_wsum008',"w+") 
dict_to_write = json.dumps(main_dicti3)
file.write(dict_to_write +"\n")
file.close()

file = open('t_wsum009',"w+") 
dict_to_write = json.dumps(main_dicti4)
file.write(dict_to_write +"\n")
file.close()

end = time.time()
print(end - start)

7318.692469835281


In [21]:
start = time.time()

main_dicti0 = {}
for key in topics:
    main_dicti0[key] = {}

main_dicti1 = {}
for key in topics:
    main_dicti1[key] = {}
    
main_dicti2 = {}
for key in topics:
    main_dicti2[key] = {}
    
main_dicti3 = {}
for key in topics:
    main_dicti3[key] = {}
    
main_dicti4 = {}
for key in topics:
    main_dicti4[key] = {}
    
    

for key in topics:

    tokenized = qe.tokenize(topics.get(key), stopWords)
    ext = qe.extend_tokens(tokenized, wv_wiki_en)
    candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=False)
    ext_candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=True)
    

    wsum11 = dr.probability_score_sum_weights(tokenized+ext, ext_candidates, text,0, -1, wv_wiki_en)
    for tupl in wsum11:
        main_dicti0[key][tupl[0]] = tupl[1]
        
    wsum106 = dr.probability_score_sum_weights(tokenized+ext, ext_candidates, text,0, 0.6, wv_wiki_en)
    for tupl in wsum106:
        main_dicti1[key][tupl[0]] = tupl[1]
        
    wsum107 = dr.probability_score_sum_weights(tokenized+ext, ext_candidates, text,0, 0.7, wv_wiki_en)
    for tupl in wsum107:
        main_dicti2[key][tupl[0]] = tupl[1]
        
    wsum108 = dr.probability_score_sum_weights(tokenized+ext, ext_candidates, text,0, 0.8, wv_wiki_en)
    for tupl in wsum108:
        main_dicti3[key][tupl[0]] = tupl[1]
        
    wsum109 = dr.probability_score_sum_weights(tokenized+ext, ext_candidates, text,0, 0.9, wv_wiki_en)
    for tupl in wsum109:
        main_dicti4[key][tupl[0]] = tupl[1]
        
        
file = open('t_wsum11',"w+") 
dict_to_write = json.dumps(main_dicti0)
file.write(dict_to_write +"\n")
file.close()
        
file = open('t_wsum106',"w+") 
dict_to_write = json.dumps(main_dicti1)
file.write(dict_to_write +"\n")
file.close()

file = open('t_wsum107',"w+") 
dict_to_write = json.dumps(main_dicti2)
file.write(dict_to_write +"\n")
file.close()

file = open('t_wsum108',"w+") 
dict_to_write = json.dumps(main_dicti3)
file.write(dict_to_write +"\n")
file.close()

file = open('t_wsum109',"w+") 
dict_to_write = json.dumps(main_dicti4)
file.write(dict_to_write +"\n")
file.close()

end = time.time()
print(end - start)

8254.605898857117


In [ ]:
22787/3600


In [22]:
import time


main_dicti0 = {}
for key in topics:
    main_dicti0[key] = {}

main_dicti1 = {}
for key in topics:
    main_dicti1[key] = {}
    
main_dicti2 = {}
for key in topics:
    main_dicti2[key] = {}
    
main_dicti3 = {}
for key in topics:
    main_dicti3[key] = {}
    
main_dicti4 = {}
for key in topics:
    main_dicti4[key] = {}
    
    

for key in topics:

   # la = time.time()
    tokenized = qe.tokenize(topics.get(key), stopWords)
    ext = qe.extend_tokens(tokenized, wv_wiki_en)
    candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=False)
    ext_candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=True)
    #ll = time.time()

#print(ll - la)


    start = time.time()
    tfidf_wsum01 = dr.tfidf_score(tokenized, text, dr.tfidf_sum_weight,0,candidates,-1,wv_wiki_en)
    for tupl in tfidf_wsum01:
        main_dicti0[key][tupl[0]] = tupl[1]
    end = time.time()
    #print(end - start)

    tfidf_wsum006 = dr.tfidf_score(tokenized, text, dr.tfidf_sum_weight,0,candidates,0.6,wv_wiki_en)
    for tupl in tfidf_wsum006:
        main_dicti1[key][tupl[0]] = tupl[1]

    tfidf_wsum007 = dr.tfidf_score(tokenized, text, dr.tfidf_sum_weight,0,candidates,0.7,wv_wiki_en)
    for tupl in tfidf_wsum007:
        main_dicti2[key][tupl[0]] = tupl[1]

    tfidf_wsum008 = dr.tfidf_score(tokenized, text, dr.tfidf_sum_weight,0,candidates,0.8,wv_wiki_en)
    for tupl in tfidf_wsum008:
        main_dicti3[key][tupl[0]] = tupl[1]

    tfidf_wsum009 = dr.tfidf_score(tokenized, text, dr.tfidf_sum_weight,0,candidates,0.9,wv_wiki_en)
    for tupl in tfidf_wsum009:
        main_dicti4[key][tupl[0]] = tupl[1]


file = open('t_tfidf_wsum01',"w+") 
dict_to_write = json.dumps(main_dicti0)
file.write(dict_to_write +"\n")
file.close()

        
file = open('t_tfidf_wsum006',"w+") 
dict_to_write = json.dumps(main_dicti1)
file.write(dict_to_write +"\n")
file.close()

file = open('t_tfidf_wsum007',"w+") 
dict_to_write = json.dumps(main_dicti2)
file.write(dict_to_write +"\n")
file.close()

file = open('t_tfidf_wsum008',"w+") 
dict_to_write = json.dumps(main_dicti3)
file.write(dict_to_write +"\n")
file.close()

file = open('t_tfidf_wsum009',"w+") 
dict_to_write = json.dumps(main_dicti4)
file.write(dict_to_write +"\n")
file.close()




In [ ]:
len(text)

In [ ]:
0.009*210158/3600 #cas za eno vprasanje 

In [23]:
start = time.time()

main_dicti0 = {}
for key in topics:
    main_dicti0[key] = {}

main_dicti1 = {}
for key in topics:
    main_dicti1[key] = {}
    
main_dicti2 = {}
for key in topics:
    main_dicti2[key] = {}
    
main_dicti3 = {}
for key in topics:
    main_dicti3[key] = {}
    
main_dicti4 = {}
for key in topics:
    main_dicti4[key] = {}
    
    

for key in topics:
    #print(key)
    tokenized = qe.tokenize(topics.get(key), stopWords)
    ext = qe.extend_tokens(tokenized, wv_wiki_en)
    candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=False)
    ext_candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=True)
    

    tfidf_wsum11 = dr.tfidf_score(tokenized+ext, text, dr.tfidf_sum_weight,0,ext_candidates,-1,wv_wiki_en)
    for tupl in tfidf_wsum11:
        main_dicti0[key][tupl[0]] = tupl[1]
        
    tfidf_wsum106 = dr.tfidf_score(tokenized+ext, text, dr.tfidf_sum_weight,0,ext_candidates,0.6,wv_wiki_en)
    for tupl in tfidf_wsum106:
        main_dicti1[key][tupl[0]] = tupl[1]
        
    tfidf_wsum107 = dr.tfidf_score(tokenized+ext, text, dr.tfidf_sum_weight,0,ext_candidates,0.7,wv_wiki_en)
    for tupl in tfidf_wsum107:
        main_dicti2[key][tupl[0]] = tupl[1]
        
    tfidf_wsum108 = dr.tfidf_score(tokenized+ext, text, dr.tfidf_sum_weight,0,ext_candidates,0.8,wv_wiki_en)
    for tupl in tfidf_wsum108:
        main_dicti3[key][tupl[0]] = tupl[1]
        
    tfidf_wsum109 = dr.tfidf_score(tokenized+ext, text, dr.tfidf_sum_weight,0,ext_candidates,0.9,wv_wiki_en)
    for tupl in tfidf_wsum109:
        main_dicti4[key][tupl[0]] = tupl[1]
        
        
file = open('t_tfidf_wsum11',"w+") 
dict_to_write = json.dumps(main_dicti0)
file.write(dict_to_write +"\n")
file.close()
        
file = open('t_tfidf_wsum106',"w+") 
dict_to_write = json.dumps(main_dicti1)
file.write(dict_to_write +"\n")
file.close()

file = open('t_tfidf_wsum107',"w+") 
dict_to_write = json.dumps(main_dicti2)
file.write(dict_to_write +"\n")
file.close()

file = open('t_tfidf_wsum108',"w+") 
dict_to_write = json.dumps(main_dicti3)
file.write(dict_to_write +"\n")
file.close()

file = open('t_tfidf_wsum109',"w+") 
dict_to_write = json.dumps(main_dicti4)
file.write(dict_to_write +"\n")
file.close()

end = time.time()
print(end - start)

KeyboardInterrupt: 

In [85]:
# testing optimal k and n for query expansion

In [87]:
main_dicti0 = {}
for key in topics:
    main_dicti0[key] = {}

main_dicti1 = {}
for key in topics:
    main_dicti1[key] = {}
    
main_dicti2 = {}
for key in topics:
    main_dicti2[key] = {}

main_dicti3 = {}
for key in topics:
    main_dicti3[key] = {}

    

for key in topics:
  
    tokenized = qe.tokenize(topics.get(key), stopWords)
    candidates10 = qe.pre_retrieval_KNN(topics.get(key), 10, wv_wiki_en, 10, stopWords,extension=False)
#     candidates20 = qe.pre_retrieval_KNN(topics.get(key), 10, wv_wiki_en, 20, stopWords,extension=False)
#     candidates30 = qe.pre_retrieval_KNN(topics.get(key), 10, wv_wiki_en, 30, stopWords,extension=False)
#    candidates = qe.pre_retrieval_KNN(topics.get(key), 10, wv_wiki_en, 10, stopWords,extension=False)
    
#     tfidf_test = dr.tfidf_score(tokenized+candidates, text, dr.tfidf_sum,0)
#     for tupl in tfidf_test:
#         main_dicti3[key][tupl[0]] = tupl[1]
    
    tfidf_cand10 = dr.tfidf_score(tokenized+candidates10, text, dr.tfidf_sum,0)
    for tupl in tfidf_cand10:
        main_dicti0[key][tupl[0]] = tupl[1]
    
#     tfidf_cand20 = dr.tfidf_score(tokenized+candidates20, text, dr.tfidf_sum,0)
#     for tupl in tfidf_cand20:
#         main_dicti1[key][tupl[0]] = tupl[1]
        
#     tfidf_cand30 = dr.tfidf_score(tokenized+candidates30, text, dr.tfidf_sum,0)
#     for tupl in tfidf_cand30:
#         main_dicti2[key][tupl[0]] = tupl[1]
        
# file = open('tfidf_test',"w+") 
# dict_to_write = json.dumps(main_dicti3)
# file.write(dict_to_write +"\n")
# file.close()    
        
file = open('tfidf_cand10',"w+") 
dict_to_write = json.dumps(main_dicti0)
file.write(dict_to_write +"\n")
file.close()
        
# file = open('tfidf_cand20',"w+") 
# dict_to_write = json.dumps(main_dicti1)
# file.write(dict_to_write +"\n")
# file.close()

# file = open('tfidf_cand30',"w+") 
# dict_to_write = json.dumps(main_dicti2)
# file.write(dict_to_write +"\n")
# file.close()

In [ ]:
# testing metrics  

import time

start = time.time()
for key in ['301','302','303','304','305']:
    print(key)
    tokenized = qe.tokenize(topics.get(key), stopWords)
    ext = qe.extend_tokens(tokenized, wv_wiki_en)
    candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=False)
    ext_candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=True)
    
    multiply0 = dr.probability_score(tokenized,text, dr.probability_multiply,0)
    multiply0 = to_dict('multiply0',key, topics, multiply0)

end = time.time()
print(end - start)

In [ ]:
    
    multiply1 = dr.probability_score(tokenized+ext,text, dr.probability_multiply,0)
    multiply1 = to_dict('multiply1',key, topics, multiply1)
    
    multiply2 = dr.probability_score(tokenized+ext+ext_candidates,text, dr.probability_multiply,0)
    multiply2 = to_dict('multiply2',key, topics, multiply2)
    
    multiply3 = dr.probability_score(tokenized+candidates,text, dr.probability_multiply,0)
    multiply3 = to_dict('multiply3',key, topics, multiply3)
    
    sum0 = dr.probability_score(tokenized,text, dr.probability_sum,0)
    sum0 = to_dict('sum0',key, topics, sum0)
    
    sum1 = dr.probability_score(tokenized+ext,text, dr.probability_sum,0)
    sum1 = to_dict('sum1',key, topics, sum1)
    
    sum2 = dr.probability_score(tokenized+ext+ext_candidates,text, dr.probability_sum,0)
    sum2 = to_dict('sum2',key, topics, sum2)
    
    sum3 = dr.probability_score(tokenized+candidates,text, dr.probability_sum,0)
    sum3 = to_dict('sum3',key, topics, sum3)
    
    for alpha in [-1, 0.6, 0.7, 0.8, 0.9]:
        wsum0 = dr.probability_score_sum_weights(tokenized, candidates, text,0, alpha, wv_wiki_en)
        wsum0 = to_dict('wsum0'+str(alpha),key, topics, wsum0)
        
        wsum1 = dr.probability_score_sum_weights(tokenized+ext, ext_candidates, text,0, alpha, wv_wiki_en)
        wsum1 = to_dict('wsum1'+str(alpha),key, topics, wsum1)
        
    tfidf_sum0 = dr.tfidf_score(tokenized, text, dr.tfidf_sum,0)
    tfidf_sum0 = to_dict('tfidf_sum0',key, topics, tfidf_sum0)
    
    tfidf_sum1 = dr.tfidf_score(tokenized+ext, text, dr.tfidf_sum,0)
    tfidf_sum1 = to_dict('tfidf_sum1',key, topics, tfidf_sum1)
    
    tfidf_sum2 = dr.tfidf_score(tokenized+ext+ext_candidates, text, dr.tfidf_sum,0)
    tfidf_sum2 = to_dict('tfidf_sum2',key, topics, tfidf_sum2)
    
    tfidf_sum3 = dr.tfidf_score(tokenized+candidates, text, dr.tfidf_sum,0)
    tfidf_sum3 = to_dict('tfidf_sum3',key, topics, tfidf_sum3)
    
    for alpha in [-1, 0.6, 0.7, 0.8, 0.9]:
        tfidf_wsum0 = dr.tfidf_score(tokenized, text, dr.tfidf_sum_weight,0,candidates,alpha,wv_wiki_en)
        tfidf_wsum0 = to_dict('tfidf_wsum0'+str(alpha),key, topics, tfidf_wsum0)
        
        tfidf_wsum1 = dr.tfidf_score(tokenized+ext, text, dr.tfidf_sum_weight,0,ext_candidates,alpha,wv_wiki_en)
        tfidf_wsum1 = to_dict('tfidf_wsum1'+str(alpha),key, topics, tfidf_wsum1)
   
        

    
    
    
    

In [ ]:
topics_results

In [ ]:
key = '305'
tokenized = qe.tokenize(topics.get(key), stopWords)
ext = qe.extend_tokens(tokenized, wv_wiki_en)
candidates = qe.pre_retrieval_KNN(topics.get(key), 5, wv_wiki_en, 10, stopWords,extension=True)
    
multiply0 = dr.probability_score(tokenized,text, dr.probability_sum,10)

In [ ]:
print(len(multiply0))
print(multiply0[0])

In [ ]:
multiply0 = to_dict('multiply0',key, topics, multiply0)

In [ ]:
multiply0.get('305')

In [ ]:
# za mac
import json
def import_file(file_path, file_name):
    file_name = {} 
    with open(file_path, 'r') as file:
            for line in file: 
                dicti = json.loads(line)
                file_name = {**file_name, **dicti}

            
            
def evaluate(file_path,file_name, qrel):
    import_file(file_path, file_name)
    evaluator = pytrec_eval.RelevanceEvaluator(qrel, {'map', 'ndcg'})
    file = open("written_"+str(file_name),"w+") 
    dict_to_write = json.dumps(evaluator.evaluate(run), indent=1)
    file.write(dict_to_write +"\n")
    file.close()

            

In [ ]:
# functions that convert data into pytrec_eval form

In [ ]:
#documents (TREC disk 4 FT)
import os
import json
import xml.etree.ElementTree as ElementTree
dir_name = "FT924"
directory = '/Users/sarab/work/try/TREC/TREC-Disk-4/FT/' + dir_name
directory_dict = {}
for filename in os.listdir(directory):
    with open('C:\\Users\\sarab\\work\\try\\TREC\\TREC-Disk-4\\FT\\'+dir_name+'\\'+filename, 'r') as f:   # Reading file
        xml = f.read()
 
    xml = '<ROOT>' + xml + '</ROOT>'   # Let's add a root tag

    root = ElementTree.fromstring(xml)

    # Simple loop through each document
    docs_json = {}
    count = 1
    for doc in root:
       # print(doc)
        #doc_json = {'DOCNO': doc.find('DOCNO').text.strip(), 'TEXT': doc.find('TEXT').text.strip()}
        try:
            doc_json = {doc.find('DOCNO').text.strip(): doc.find('TEXT').text.strip()}
        except:
            print(doc.find('DOCNO').text.strip(), count)
            count = count +1
            doc_json = {doc.find('DOCNO').text.strip(): 'empty'}
        docs_json.update(doc_json)
    directory_dict = {**directory_dict, **docs_json}
    continue
    
file = open("written_"+dir_name,"w+") 
dict_to_write = json.dumps(directory_dict)
file.write(dict_to_write +"\n")
file.close()
    

In [ ]:
#documents (TREC disk 4 FT)
import os
import json
import xml.etree.ElementTree as ElementTree
dir_name = "01"
directory = '/Users/sarab/work/try/TREC/TREC-Disk-4/FR94/' + dir_name
directory_dict = {}
for filename in os.listdir(directory):
    with open('C:\\Users\\sarab\\work\\try\\TREC\\TREC-Disk-4\\FR94\\'+dir_name+'\\'+filename, 'r') as f:   # Reading file
        xml = f.read()
 
    xml = '<ROOT>' + xml + '</ROOT>'   # Let's add a root tag

    root = ElementTree.fromstring(xml)

    # Simple loop through each document
    docs_json = {}
    count = 1
    for doc in root:
       # print(doc)
        #doc_json = {'DOCNO': doc.find('DOCNO').text.strip(), 'TEXT': doc.find('TEXT').text.strip()}
        try:
            doc_json = {doc.find('DOCNO').text.strip(): doc.find('TEXT').text.strip()}
        except:
            print(doc.find('DOCNO').text.strip(), count)
            count = count +1
            doc_json = {doc.find('DOCNO').text.strip(): 'empty'}
        docs_json.update(doc_json)
    directory_dict = {**directory_dict, **docs_json}
    continue
    
file = open("written_"+dir_name,"w+") 
dict_to_write = json.dumps(directory_dict)
file.write(dict_to_write +"\n")
file.close()

In [ ]:
#documents (TREC disk 4 and 5)
import os
import json
import xml.etree.ElementTree as ElementTree
dir_name = "F"
directory = '/Users/sarab/work/try/TREC/TREC-Disk-5/FBIS/' + dir_name
directory_dict = {}
for filename in os.listdir(directory):
    print(filename)
    with open('C:\\Users\\sarab\\work\\try\\TREC\\TREC-Disk-5\\FBIS\\'+dir_name+'\\'+filename, 'r') as f:   # Reading file
        xml = f.read()
    parser = ElementTree.XMLParser(encoding="utf-8")
    
    xml = '<ROOT>' + xml + '</ROOT>'   # Let's add a root tag
    
    root = ElementTree.fromstring(xml, parser = parser)

    # Simple loop through each document
    docs_json = {}
    for doc in root:
        #doc_json = {'DOCNO': doc.find('DOCNO').text.strip(), 'TEXT': doc.find('TEXT').text.strip()}
        doc_json = {doc.find('DOCNO').text.strip(): doc.text.strip()}
        docs_json.update(doc_json)
    directory_dict = {**directory_dict, **docs_json}
    continue
    
file = open("written_"+dir_name,"w+") 
dict_to_write = json.dumps(directory_dict)
file.write(dict_to_write +"\n")
file.close()
    

In [ ]:
import json
import xml.etree.ElementTree as ElementTree
dir_name = "L"
directory = '/Users/sarab/work/try/TREC/TREC-Disk-5/LATIMES/' + dir_name
directory_dict = {}
count = 1
for filename in os.listdir(directory):
    with open('C:\\Users\\sarab\\work\\try\\TREC\\TREC-Disk-5\\LATIMES\\'+dir_name+'\\'+filename, 'r') as f:   # Reading file
        xml = f.read()
 
    xml = '<ROOT>' + xml + '</ROOT>'   # Let's add a root tag

    root = ElementTree.fromstring(xml)

    # Simple loop through each document
    docs_json = {}
    
    for doc in root:
        print(root[0][1].text)
        #doc_json = {'DOCNO': doc.find('DOCNO').text.strip(), 'TEXT': doc.find('TEXT').text.strip()}
        
        doc_json = {doc.find('DOCNO').text.strip(): doc.text} #doc.attrib}
        docs_json.update(doc_json)
    directory_dict = {**directory_dict, **docs_json}
    continue
    
file = open("written_"+dir_name,"w+") 
dict_to_write = json.dumps(directory_dict)
file.write(dict_to_write +"\n")
file.close()

In [ ]:
#delujoca
import json
import xml.etree.ElementTree as ElementTree
dir_name = "L"
directory = '/Users/sarab/work/try/TREC/TREC-Disk-5/LATIMES/' + dir_name
directory_dict = {}
count = 1
for filename in os.listdir(directory):
    with open('C:\\Users\\sarab\\work\\try\\TREC\\TREC-Disk-5\\LATIMES\\'+dir_name+'\\'+filename, 'r') as f:   # Reading file
        xml = f.read()
 
    xml = '<ROOT>' + xml + '</ROOT>'   # Let's add a root tag

    root = ElementTree.fromstring(xml)
    

    # Simple loop through each document
    docs_json = {}
    
    for doc in root:
        
        #doc_json = {'DOCNO': doc.find('DOCNO').text.strip(), 'TEXT': doc.find('TEXT').text.strip()}
        
        doc_json = {doc.find('DOCNO').text.strip(): ' '.join(doc.itertext()).replace('\n','')} #doc.attrib}
        docs_json.update(doc_json)
    directory_dict = {**directory_dict, **docs_json}
    continue


file = open("written_"+dir_name,"w+") 
dict_to_write = json.dumps(directory_dict)
file.write(dict_to_write +"\n")
file.close()

In [ ]:
#delujoca
import json
import xml.etree.ElementTree as ElementTree
dir_name = "F"
directory = '/Users/sarab/work/try/TREC/TREC-Disk-5/FBIS/' + dir_name
directory_dict = {}
for filename in os.listdir(directory):
    print(filename)
    with open('C:\\Users\\sarab\\work\\try\\TREC\\TREC-Disk-5\\FBIS\\'+dir_name+'\\'+filename, 'r') as f:   # Reading file Reading file
        xml = f.read()
 
    xml = '<ROOT>' + xml + '</ROOT>'   # Let's add a root tag

    root = ElementTree.fromstring(xml)
    

    # Simple loop through each document
    docs_json = {}
    
    for doc in root:
        
        #doc_json = {'DOCNO': doc.find('DOCNO').text.strip(), 'TEXT': doc.find('TEXT').text.strip()}
        
        doc_json = {doc.find('DOCNO').text.strip(): ' '.join(doc.itertext()).replace('\n','')} #doc.attrib}
        docs_json.update(doc_json)
    directory_dict = {**directory_dict, **docs_json}
    continue


file = open("written_"+dir_name,"w+") 
dict_to_write = json.dumps(directory_dict)
file.write(dict_to_write +"\n")
file.close()

In [ ]:
len(directory_dict)

In [ ]:
# questions (300 - 450 ad hoc)
import re
directory = '/Users/sarab/work/try/TREC/topics/' 
#directory_list = []
question_dictionary = {}
for filename in os.listdir(directory):

    with open('C:\\Users\\sarab\\work\\try\\TREC\\topics\\'+filename) as f:
        file = f.read()
        numbers = re.findall('<num>\sNumber:\s\d+',file,re.MULTILINE)
        questions = re.findall('<title>\s(.*?)<desc>',file,re.DOTALL)
        for i in range(len(numbers)):
            number = re.search('\d+', numbers[i])
            number = number.group(0)
            question = questions[i].replace('\n','')
            question_dictionary.update({number: question})

file = open("titles_topics","w+") 
dict_to_write = json.dumps(question_dictionary)
file.write(dict_to_write)
file.close()

In [ ]:
# qrels for questions 300 - 450

import re
directory = '/Users/sarab/work/try/TREC/qrels/' 
tuple_list= []
for filename in os.listdir(directory):
    with open('C:\\Users\\sarab\\work\\try\\TREC\\qrels\\'+filename) as f:
        file = f.read()
       # questions = re.findall(r'(\d+)\s0\s.+\s\d\n',file)
        documents = re.findall(r'\d+\s0\s(.+)\s\d\n',file)
        relavances = re.findall(r'(\d+)\s0\s.+\s(\d)\n',file)
        if len(relavances) == len(documents) :
            for i in range(len(relavances)):
                tuple_list.append((relavances[i][0], documents[i], relavances[i][1]))
                
        else:
            print('Error')
            
#write in form for pytrec_eval:
main_dict = {}
for nb in range(301,451):
    main_dict[nb] = {}
for tupl in tuple_list:
    main_dict[int(float(tupl[0]))][tupl[1]] = tupl[2]

file = open("qrels","w+") 
dict_to_write = json.dumps(main_dict)
file.write(dict_to_write)
file.close()
    


            


In [ ]:
class Decoder(json.JSONDecoder):
    def decode(self, s):
        result = super().decode(s)  # result = super(Decoder, self).decode(s) for Python 2.x
        return self._decode(result)

    def _decode(self, o):
        if isinstance(o, str):
            try:
                return int(o)
            except ValueError:
                return o
        elif isinstance(o, dict):
            return {k: self._decode(v) for k, v in o.items()}
        elif isinstance(o, list):
            return [self._decode(v) for v in o]
        else:
            return o

In [ ]:
def import_file(file_path):
    file_dict = {}
    with open(file_path, 'r') as file:
        for line in file:
            dicti = json.loads(line, cls=Decoder)
            file_dict = {**file_dict, **dicti}
    return file_dict


In [ ]:
qrel = import_file("/Users/sarab/work/try/TREC/processed_data/qrels")

In [ ]:
LATIMES = import_file("/Users/sarab/work/try/enviroLENS/word-embeddings/notebooks/written_L")

In [ ]:
LATIMES.get("LA010189-0001")

In [ ]:
qrel.get("301").get('CR93E-10279')


In [ ]:
# analysing TREC results

In [ ]:
import ast
directory = "/Users/sarab/work/try/TREC/processed_data/eval_results"
map_results = {}

for filename in os.listdir(directory):
    map_results[filename] = {}
    q301_350 = 0
    q351_400 = 0
    q401_450 = 0
    map_val = 0
    count = 0
    c1 = 0
    c2 = 0
    c3 = 0

    with open(directory + '/' + filename, 'r') as file:
        results = json.loads(file.read(), cls=Decoder)
    for key,value in results.items():
        key = int(key)
        val = value.get("map")
        if 301 <= key <=350:
            c1 += 1
            q301_350 = q301_350 + val
                
        elif 351 <= key <=400:
            c2 += 1
            q351_400 = q351_400 + val
            
        elif 401 <= key <=450:
            c3 += 1
            q401_450 = q401_450 + val
            
        else:
            print('error')
            
        map_val = map_val + val
        count += 1
    map_final = map_val / count
    q301_350 = q301_350 / c1
    q351_400 = q351_400 / c2
    q401_450 = q401_450 / c3
    map_results[filename]['q301_450'] = map_final
    map_results[filename]['q301_350'] = q301_350
    map_results[filename]['q351_400'] = q401_450
    map_results[filename]['q401_450'] = q401_450
    if c1 !=50 or c2 !=50 or c1 !=50 or count!=150 :
        print('error_c')
    
    


In [ ]:
import pandas
df = pandas.DataFrame.from_dict(map_results,'index')
df.sort_values(by=['q301_450'], ascending=False)